### Initial Setup and Configuration:
- Changing and Verifying the Working Directory for Project Setup
- This section ensures that your working directory is set to the project root.

In [1]:
import sys
import os

# Check current working directory
current_dir = os.getcwd()
print(f"Current Directory: {current_dir}")

# Change to your project root directory
project_root = '/home/razvansavin/Proiecte/flexiai'
os.chdir(project_root)
print(f"Changed Directory to: {os.getcwd()}")

# Add project root directory to sys.path
sys.path.append(project_root)
print(f"Project root added to sys.path")

Current Directory: /home/razvansavin/Proiecte/flexiai/flexiai/examples
Changed Directory to: /home/razvansavin/Proiecte/flexiai
Project root added to sys.path


### Import necessary modules:
- Import required modules and set up logging.

In [2]:
import json
import logging
from flexiai.core.flexiai_client import FlexiAI
from flexiai.config.logging_config import setup_logging
from flexiai.core.utils.helpers import show_json, print_messages_as_json, print_run_details

# Setup logging using the predefined configuration
setup_logging(root_level=logging.DEBUG, file_level=logging.DEBUG, console_level=logging.ERROR)

# Initialize FlexiAI
flexiai = FlexiAI()



### Define the assistant ID:
- Define the assistant ID that will be used for interactions.

In [3]:
# Use the given assistant ID
assistant_id = 'asst_AWAVO511bAbTVEdOvLNWitoT'  # Replace with the actual assistant ID


### Create a new thread:
- Create a new thread to start a conversation.

In [4]:
thread = flexiai.create_thread()
thread_id = thread.id
print(f"Thread Created with ID: {thread_id}")

Thread Created with ID: thread_RjWW64IXOOki6AujRbCk5e8I



### Variable to store all messages:
- Initialize variables to store messages and keep track of processed message IDs.

In [5]:
all_messages = []
seen_message_ids = set()

### Add a user message and print the message:
- Add a message from the user to the thread and print the message details.

In [6]:
user_message = "Tell me about the Eiffel Tower."
message = flexiai.add_user_message(thread_id=thread_id, user_message=user_message)
show_json(message)

# Store the message
all_messages.append({"role": "user", "content": user_message})
seen_message_ids.add(message.id)


{
    "id": "msg_WvMYCuaHf6WdFsibxhALsi1S",
    "assistant_id": null,
    "attachments": [],
    "completed_at": null,
    "content": [
        {
            "text": {
                "annotations": [],
                "value": "Tell me about the Eiffel Tower."
            },
            "type": "text"
        }
    ],
    "created_at": 1720357945,
    "incomplete_at": null,
    "incomplete_details": null,
    "metadata": {},
    "object": "thread.message",
    "role": "user",
    "run_id": null,
    "status": null,
    "thread_id": "thread_RjWW64IXOOki6AujRbCk5e8I"
}



### Create a run to get the assistant's response:
- Create a run to get the response from the assistant.

In [7]:
run = flexiai.create_run(assistant_id=assistant_id, thread_id=thread_id)
print_run_details(run)


{
    "id": "run_7NjGKENb9GqI3HfqPIrv9J6P",
    "assistant_id": "asst_AWAVO511bAbTVEdOvLNWitoT",
    "cancelled_at": null,
    "completed_at": 1720357949,
    "created_at": 1720357946,
    "expires_at": null,
    "failed_at": null,
    "incomplete_details": null,
    "instructions": "### Instructions for Assistant Alina\n\nAs Assistant Alina, you are a versatile assistant dedicated to offering exceptional support. You are capable of executing YouTube searches on user request. The following instructions outline the steps you should follow to provide efficient and effective assistance. You will execute searches only after user confirmation.\n\n### Workflow\n\n#### General Inquiries\n\n1. **Handling General Inquiries**:\n   - Address user queries to the best of your ability, providing information, support, and resources as needed.\n\n2. **Using the YouTube Search Function**:\n   - When a user requests assistance with finding video content on YouTube, use the YouTube search functionality.\

### Retrieve and store the assistant's response:
- Retrieve messages from the thread and store them in a list.

In [8]:
messages = flexiai.retrieve_messages(thread_id=thread_id, order='desc', limit=2)

for msg in messages:
    if msg['message_id'] not in seen_message_ids:
        all_messages.append(msg)
        seen_message_ids.add(msg['message_id'])

# Print all messages
for msg in all_messages:
    role = "🤖 Assistant" if msg['role'] == "assistant" else "🧑 You"
    print(f"{role}: {msg['content']}")


🧑 You: Tell me about the Eiffel Tower.
🤖 Assistant: The Eiffel Tower is a famous landmark in Paris, France. Here are some key points about the Eiffel Tower:

1. **Iconic Structure**: The Eiffel Tower is an iconic iron lattice tower located on the Champ de Mars in Paris. It is one of the most recognizable structures in the world.

2. **Construction**: It was designed by Gustave Eiffel and his construction company. The tower was built as the entrance arch for the 1889 World's Fair, held in Paris to celebrate the 100th anniversary of the French Revolution.

3. **Height**: The Eiffel Tower stands at a height of 1,063 feet (324 meters) and was the tallest man-made structure in the world until the completion of the Chrysler Building in New York in 1930.

4. **Visitors**: It is a major tourist attraction, with millions of visitors ascending the tower each year to enjoy panoramic views of Paris from its observation decks.

5. **Lighting**: The Eiffel Tower is illuminated by thousands of sparkl

### Add another user message and print the message:
- Add another message from the user to the thread and print the message details.
- This message will trigger the `search_youtube` function.

In [9]:
user_message = "I would like to search on YouTube these keywords: Eiffel Tower."
message = flexiai.add_user_message(thread_id=thread_id, user_message=user_message)
print(message)

# Store the message
all_messages.append({"role": "user", "content": user_message})
seen_message_ids.add(message.id)


Message(id='msg_xEgZVZiIj1o4GZ7mD6zR1WkR', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='I would like to search on YouTube these keywords: Eiffel Tower.'), type='text')], created_at=1720357950, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_RjWW64IXOOki6AujRbCk5e8I')



### Create another run to get the assistant's response:
- Create another run to get the assistant's response to the new message.
- The `create_run` function can handle `requires_action` and will trigger a search on YouTube with the `search_youtube` function if it is mapped in `user_flexiai_rag/user_function_mapping.py` and stored in `user_flexiai_rag/user_task_manager.py`.


In [10]:
run = flexiai.create_run(assistant_id=assistant_id, thread_id=thread_id)
show_json(run)

{
    "id": "run_9hGPEeDrq56IeO3fGUJ82mwc",
    "assistant_id": "asst_AWAVO511bAbTVEdOvLNWitoT",
    "cancelled_at": null,
    "completed_at": 1720357954,
    "created_at": 1720357951,
    "expires_at": null,
    "failed_at": null,
    "incomplete_details": null,
    "instructions": "### Instructions for Assistant Alina\n\nAs Assistant Alina, you are a versatile assistant dedicated to offering exceptional support. You are capable of executing YouTube searches on user request. The following instructions outline the steps you should follow to provide efficient and effective assistance. You will execute searches only after user confirmation.\n\n### Workflow\n\n#### General Inquiries\n\n1. **Handling General Inquiries**:\n   - Address user queries to the best of your ability, providing information, support, and resources as needed.\n\n2. **Using the YouTube Search Function**:\n   - When a user requests assistance with finding video content on YouTube, use the YouTube search functionality.\

### Retrieve and print the last messages:
- Retrieve the latest messages from the thread and print them.

In [11]:
messages = flexiai.retrieve_messages(thread_id=thread_id, order='desc', limit=2)

for msg in messages:
    if msg['message_id'] not in seen_message_ids:
        all_messages.append(msg)
        seen_message_ids.add(msg['message_id'])

# Print all messages
for msg in all_messages:
    role = "🤖 Assistant" if msg['role'] == "assistant" else "🧑 You"
    print(f"{role}: {msg['content']}")


🧑 You: Tell me about the Eiffel Tower.
🤖 Assistant: The Eiffel Tower is a famous landmark in Paris, France. Here are some key points about the Eiffel Tower:

1. **Iconic Structure**: The Eiffel Tower is an iconic iron lattice tower located on the Champ de Mars in Paris. It is one of the most recognizable structures in the world.

2. **Construction**: It was designed by Gustave Eiffel and his construction company. The tower was built as the entrance arch for the 1889 World's Fair, held in Paris to celebrate the 100th anniversary of the French Revolution.

3. **Height**: The Eiffel Tower stands at a height of 1,063 feet (324 meters) and was the tallest man-made structure in the world until the completion of the Chrysler Building in New York in 1930.

4. **Visitors**: It is a major tourist attraction, with millions of visitors ascending the tower each year to enjoy panoramic views of Paris from its observation decks.

5. **Lighting**: The Eiffel Tower is illuminated by thousands of sparkl

---
---

### Example to show different formats for retrieving messages:
- This section demonstrates two different formats for retrieving messages from a thread using FlexiAI.


#### Retrieve Messages:
- The `retrieve_messages` function fetches messages in a list of formatted dictionaries.
- The `order='desc'` argument specifies that the messages are retrieved in descending order (newest first).
- `limit=20` specifies the maximum number of messages to retrieve.
- The retrieved messages are printed to show their format.


In [12]:
print(100*'=')
messages = flexiai.retrieve_messages(thread_id=thread_id, order='desc', limit=20)

# Pretty print the messages
pretty_messages = json.dumps(messages, indent=4)
print(pretty_messages)
print(100*'=')

[
    {
        "message_id": "msg_WvMYCuaHf6WdFsibxhALsi1S",
        "role": "user",
        "content": "Tell me about the Eiffel Tower."
    },
    {
        "message_id": "msg_bLMZkJDeM5UrEAM6WHmc2r8O",
        "role": "assistant",
        "content": "The Eiffel Tower is a famous landmark in Paris, France. Here are some key points about the Eiffel Tower:\n\n1. **Iconic Structure**: The Eiffel Tower is an iconic iron lattice tower located on the Champ de Mars in Paris. It is one of the most recognizable structures in the world.\n\n2. **Construction**: It was designed by Gustave Eiffel and his construction company. The tower was built as the entrance arch for the 1889 World's Fair, held in Paris to celebrate the 100th anniversary of the French Revolution.\n\n3. **Height**: The Eiffel Tower stands at a height of 1,063 feet (324 meters) and was the tallest man-made structure in the world until the completion of the Chrysler Building in New York in 1930.\n\n4. **Visitors**: It is a major


#### Retrieve Message Objects:
- The `retrieve_message_object` function fetches the entire message objects from the thread.
- The `order='asc'` argument specifies that the messages are retrieved in ascending order (oldest first).
- `limit=20` specifies the maximum number of message objects to retrieve.
- The retrieved message objects are printed to show their detailed structure.

In [13]:
messages = flexiai.retrieve_message_object(thread_id=thread_id, order='asc', limit=20)
print_messages_as_json(messages)


[
    {
        "id": "msg_WvMYCuaHf6WdFsibxhALsi1S",
        "assistant_id": null,
        "attachments": [],
        "completed_at": null,
        "content": [
            {
                "text": {
                    "annotations": [],
                    "value": "Tell me about the Eiffel Tower."
                },
                "type": "text"
            }
        ],
        "created_at": 1720357945,
        "incomplete_at": null,
        "incomplete_details": null,
        "metadata": {},
        "object": "thread.message",
        "role": "user",
        "run_id": null,
        "status": null,
        "thread_id": "thread_RjWW64IXOOki6AujRbCk5e8I"
    },
    {
        "id": "msg_bLMZkJDeM5UrEAM6WHmc2r8O",
        "assistant_id": "asst_AWAVO511bAbTVEdOvLNWitoT",
        "attachments": [],
        "completed_at": null,
        "content": [
            {
                "text": {
                    "annotations": [],
                    "value": "The Eiffel Tower is a famous l